In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import pickle

In [114]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
#RandomForestClassifier()

In [115]:
df_breaks = pd.read_pickle('..//data//simulated_breaks_ix.pickle')
df_returns = pd.read_pickle('..//data//simulated_returns.pickle')
df_tangency_portfolio = pd.read_pickle('..//data//simulated_tangency_portfolios.pickle')
df_highest_sharp = pd.read_pickle('..//data//highest_shape_ratios.pickle')

In [158]:
def get_val(row):
    for ix, val in enumerate(row):
        if val == 1:
            return ix
        
        
res = df_highest_sharp.apply(get_val, axis=1)

In [4]:
def max_val(x, max_val=5):
    """Function made for trimming weights of portfolio"""
    
    if abs(x) > max_val:
        return np.sign(x) * max_val
    return x

In [5]:
train_frac = 0.9
data_length = len(df_returns)
split_ix = int(train_frac * data_length)

# column removed for identification pruposes
skip_col = df_tangency_portfolio.columns[-1]
df_tangency = df_tangency_portfolio.drop(skip_col, axis=1)

# capping weird weigths
df_tangency_masked = pd.DataFrame(df_tangency)
for col in df_tangency_masked.columns:
    df_tangency_masked[col] = df_tangency_masked[col].apply(max_val)

In [6]:
dfs = [df_returns]

n_shifts = 4

for i in range(1,4+1):

    df_shift = df_returns.shift(i)
    df_shift.rename({key : key + f':skip={i}' for key in df_returns.columns}, axis=1, inplace=True)
    dfs.append(df_shift)

In [7]:
df_features = pd.concat(dfs, axis=1)
df_features.head()

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_2:skip=4,s_3:skip=4,s_4:skip=4,s_5:skip=4,s_6:skip=4,s_7:skip=4,s_8:skip=4,s_9:skip=4,s_10:skip=4,s_11:skip=4
0,-0.007478,0.023819,0.009998,-0.000506,0.016793,0.008032,0.011506,0.018415,0.000949,0.047258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.025152,0.030963,0.009738,0.012595,0.006293,0.000239,-0.010242,0.016026,-0.012189,-0.027691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.021323,0.015792,0.012332,0.024294,0.017750,0.009980,0.017866,0.022732,0.020637,0.004990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.002132,-0.007434,0.007005,0.012630,0.006596,-0.005513,-0.005403,0.002853,-0.016371,-0.022594,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.014248,-0.002495,-0.005517,0.004141,-0.006622,-0.009481,0.002114,-0.010759,0.001401,0.002718,...,0.023819,0.009998,-0.000506,0.016793,0.008032,0.011506,0.018415,0.000949,0.047258,-0.009535


In [8]:
df_features = df_features.dropna()

In [165]:
df_target = res.iloc[n_shifts : ,]

In [167]:
df_features.shape, df_target.shape

((1999996, 55), (1999996,))

In [168]:
x_train = df_features[0:split_ix]
x_test = df_features[split_ix:]

In [169]:
y_train = df_target[0:split_ix]
y_test = df_target[split_ix:]

In [170]:
def choose_indices(start, end, modulo):
    return list(filter(lambda x: False if x % modulo==0 else True, range(start, end + 1)))
    
def create_subsample(x, y, modulo=500):
    start, end = min(x.index), max(x.index)
    ixs = choose_indices(start, end, modulo)
    #print(ixs)
    return x.drop(ixs), y.drop(ixs)

In [184]:
_x, _y = create_subsample(x_train, y_train, modulo=50)

In [192]:
rf = RandomForestClassifier(n_estimators=200, max_depth=3)

In [193]:
rf.fit(_x, _y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [194]:
res = rf.predict(_x)

In [195]:
rf.score(_x, _y)

0.12052777777777778

In [196]:
pd.DataFrame(res - np.array(_y))

,0
0,4
1,6
2,6
3,6
4,6
5,6
6,6
7,8
8,6
9,6


In [190]:
_test = rf.predict(x_test)

In [197]:
rf.score(x_test, y_test)

0.08088661773235464

In [180]:
pd.DataFrame({'pred': _test, 'true': y_test})

,pred,true
1800004,10,9
1800005,0,9
1800006,10,9
1800007,4,9
1800008,8,9
1800009,8,9
1800010,9,9
1800011,6,9
1800012,10,9
1800013,5,9


In [147]:
from sklearn.datasets import make_classification


In [148]:
X, y = make_classification(n_samples=1000, n_features=4,
                  n_informative=2, n_redundant=0,
                      random_state=0, shuffle=False)

In [151]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [152]:
df_target

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,s_11
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


0          6
1          6
2          6
3          6
4          6
          ..
1999995    5
1999996    5
1999997    5
1999998    5
1999999    5
Length: 2000000, dtype: int64